In [130]:
# Jenny branch
import numpy as np

import datetime

import tensorflow as tf 

from sklearn import linear_model, ensemble, tree, svm, naive_bayes
from scipy.stats import uniform as sp_rand

import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Flatten, BatchNormalization, GlobalAveragePooling1D
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV

from collections import Counter



BORDER = "===================================================================================="

In [82]:
def load_data(filename, skiprows = 1):
    """
    Function loads data stored in the file filename and returns it as a numpy ndarray.
    
    Inputs:
        filename: given as a string.
        
    Outputs:
        Data contained in the file, returned as a numpy ndarray
    """
    return np.loadtxt(filename, skiprows=skiprows, delimiter=' ')

In [83]:
def WritePredictionOut(modelName, prediction):
    print(prediction)
    print("prediction shape is: {}".format(prediction.shape))

    output = "Id,Prediction\n"
    for i in range(prediction.shape[0]):
        output = output + ("{0},{1}\n".format(i + 1, prediction[i].astype(int)))

        
    now = datetime.datetime.now();
        
    filename = "{}_{}.{}.{}_{}.{}.{}_predictions.csv".format(modelName, now.year, now.month, now.day, now.hour, now.minute, now.second)
        
    file = open(filename,'w') 
    file.write(output)
    file.close()

In [193]:
def ClassifyWithNeuralNetwork(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using our neural network implementation
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    if (verbose == 1):
        print('\n{}\nNEURAL NETWORK\n{}\n'.format(BORDER, BORDER))

    # Converting Y values to one hot vector
    num_classes = 2
    Y_train = keras.utils.to_categorical(Y_train, num_classes=num_classes)
    
    # Input size
    n_train = X_train.shape[0]
    n_words = X_train.shape[1]
    n_test = Y_train.shape[0]
    
    # Layer set up.
    model = Sequential()
    
    model.add(Dense(128, input_shape=(n_words, )))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    ## Printing a summary of the layers and weights in your model
    if (verbose == 1):
        model.summary()

    ## In the line below we have specified the loss function as 'mse' (Mean Squared Error) because in the above code we did not one-hot encode the labels.
    ## In your implementation, since you are one-hot encoding the labels, you should use 'categorical_crossentropy' as your loss.
    ## You will likely have the best results with RMS prop or Adam as your optimizer.  In the line below we use Adadelta
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

    fit = model.fit(X_train, Y_train, batch_size=256, epochs=4, verbose=verbose, validation_split=0.0)
    
    ## Printing the accuracy of our model, according to the loss function specified in model.compile above
    score = model.evaluate(X_train, Y_train, verbose=0)
    if (verbose == 1):
        print('Training score:', score[0])
        print('Training accuracy:', score[1])

    prediction = model.predict(X_test, verbose=verbose)

    zeros = prediction[:, 0]
    ones  = prediction[:, 1]

    prediction = (zeros < ones).astype(int)

    return model, prediction, score[1]

In [194]:
def ClassifyWithConvolutionNetwork(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using our neural network implementation
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    if (verbose == 1):
        print('\n{}\nNEURAL NETWORK\n{}\n'.format(BORDER, BORDER))

    # Converting Y values to one hot vector
    num_classes = 2
    Y_train = keras.utils.to_categorical(Y_train, num_classes=num_classes)
    
    # Input size
    n_train = X_train.shape[0]
    n_words = X_train.shape[1]
    n_test = Y_train.shape[0]
    
    
    # For 1D Convolution, change dimension of input 
    X_train = np.expand_dims(X_train, axis=2)
    X_test = np.expand_dims(X_test, axis=2)
    
        
    ## Create your own model here given the constraints in the problem
    model = Sequential()

    model.add(Conv1D(128, 3, padding="same", input_shape=(n_words, 1, )))

    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    ## Once you one-hot encode the data labels, the line below should be predicting probabilities of each of the 2 classes
    model.add(Dense(2))
    model.add(Activation('softmax'))

    ## Printing a summary of the layers and weights in your model
    if (verbose == 1):
        model.summary()

    ## In the line below we have specified the loss function as 'mse' (Mean Squared Error) because in the above code we did not one-hot encode the labels.
    ## In your implementation, since you are one-hot encoding the labels, you should use 'categorical_crossentropy' as your loss.
    ## You will likely have the best results with RMS prop or Adam as your optimizer.  In the line below we use Adadelta
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

    fit = model.fit(X_train, Y_train, batch_size=128, epochs=3, verbose=verbose, validation_split=0.0)
    
    ## Printing the accuracy of our model, according to the loss function specified in model.compile above
    score = model.evaluate(X_train, Y_train, verbose=0)
    
    if (verbose == 1):
        print('Training score:', score[0])
        print('Training accuracy:', score[1])

    prediction = model.predict(X_test, verbose=verbose)

    zeros = prediction[:, 0]
    ones  = prediction[:, 1]

    prediction = (zeros < ones).astype(int)

    return model, prediction, score[1]

In [207]:
def ClassifyWithLogisticRegression(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nLOGISTIC REGRESSION\n{}\n'.format(BORDER, BORDER))
    
    model = linear_model.LogisticRegression(penalty='l1', solver='saga')
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [87]:
def ClassifyWithLinearRegression(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nLINEAR REGRESSION\n{}\n'.format(BORDER, BORDER))
    
    model = linear_model.LinearRegression()
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [88]:
def ClassifyWithAdaBoost(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """

    if (verbose == 1):
        print('\n{}\nADABOOST\n{}\n'.format(BORDER, BORDER))
    
    model = ensemble.AdaBoostClassifier()
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [89]:
def ClassifyWithDecisionTree(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nDECISION TREE\n{}\n'.format(BORDER, BORDER))
        
    model = tree.DecisionTreeClassifier()
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [90]:
def ClassifyWithRandomForest(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nRANDOM FOREST\n{}\n'.format(BORDER, BORDER))
        
    model = ensemble.RandomForestClassifier()
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [220]:
def ClassifyWithSVM(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nSVM\n{}\n'.format(BORDER, BORDER))
        
    model = svm.LinearSVC(penalty='l1', verbose=verbose, dual=False, C=0.1)
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [92]:
def ClassifyWithSVC(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nSVC\n{}\n'.format(BORDER, BORDER))
        
    model = svm.SVC(kernel="rbf", C=0.03, verbose=verbose)
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [229]:
def ClassifyWithBernoulliNB(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nBernoulli Naive Bayes\n{}\n'.format(BORDER, BORDER))
        
    model = naive_bayes.BernoulliNB(alpha=0.01)
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [94]:
def ClassifyWithMultinomialNB(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\n Multinomial Naive Bayes\n{}\n'.format(BORDER, BORDER))
        
    model = naive_bayes.MultinomialNB(alpha=.01)
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [95]:
def ClassifyWithBlending(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
    """
    
    if (verbose == 1):
        print('\n{}\nBlending\n{}\n'.format(BORDER, BORDER))
        
    _, pred_NN, _ = ClassifyWithNeuralNetwork(X_train, Y_train, X_test, verbose=verbose)
    _, pred_CN, _ = ClassifyWithConvolutionNetwork(X_train, Y_train, X_test, verbose=verbose)
    _, pred_LR, _ = ClassifyWithLogisticRegression(X_train, Y_train, X_test, verbose=verbose)
    _, pred_SVM, _ = ClassifyWithSVM(X_train, Y_train, X_test, verbose=verbose)
    _, pred_NB, _ = ClassifyWithBernoulliNB(X_train, Y_train, X_test, verbose=verbose)
        
    # Combine prediction into list
    preds = []
    for i in range(len(pred_NN)):
        preds.append((pred_NN[i], pred_CN[i], pred_LR[i], pred_SVM[i], pred_NB[i]))
            
    
    # Final prediction
    Y_pred = np.array([])
    for i in range(len(preds)):
        lst = Counter(preds[i]).most_common()
        Y_pred = np.append(Y_pred, int(lst[0][0]))
    
    return Y_pred

In [141]:
def ClassifyWithStacking(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
    """
    
    if (verbose == 1):
        print('\n{}\nBlending\n{}\n'.format(BORDER, BORDER))
        
    _, pred1, _ = ClassifyWithNeuralNetwork(X_train, Y_train, X_test, verbose=verbose)
    _, pred2, _ = ClassifyWithNeuralNetwork(X_train, Y_train, X_test, verbose=verbose)
    _, pred3, _ = ClassifyWithNeuralNetwork(X_train, Y_train, X_test, verbose=verbose)
    _, pred4, _ = ClassifyWithNeuralNetwork(X_train, Y_train, X_test, verbose=verbose)
    _, pred5, _ = ClassifyWithNeuralNetwork(X_train, Y_train, X_test, verbose=verbose)
        
    # Combine prediction into list
    preds = []
    for i in range(len(pred1)):
        preds.append((pred1[i], pred2[i], pred3[i], pred4[i], pred5[i]))
            
    
    # Final prediction
    Y_pred = np.array([])
    for i in range(len(preds)):
        lst = Counter(preds[i]).most_common()
        Y_pred = np.append(Y_pred, int(lst[0][0]))
    
    return Y_pred

In [97]:
def CrossValidation_NeuralNetwork(X_train, Y_train, X_test, nFold, verbose=0):
    
    kf = KFold(n_splits = nFold)
    inds = [ind for ind in kf.split(X_train, Y_train)]
        
    total_train_acc = []
    total_val_acc = []
    
     # Converting Y values to one hot vector
    num_classes = 2
    Y = keras.utils.to_categorical(Y_train, num_classes=num_classes)
    
    # perform 5-fold validation
    for i in range(0,nFold):
        traini, vali = inds[i]
        model, prediction, accuracy = ClassifyWithNeuralNetwork(X_train[traini], Y_train[traini], X_test)
        
        # Compute accuracy.
        val_acc = model.evaluate(X_train[vali], Y[vali], verbose=verbose)
            
        total_train_acc = np.append(total_train_acc, accuracy)
        total_val_acc = np.append(total_val_acc, val_acc[1])

    print(BORDER)
    print("CROSS VALIDATION: ClassifyWithNeuralNetwork")
    print(BORDER)
    print("training accuracy", total_train_acc)
    print("val accuracy", total_val_acc)
    print("average training accuracy", np.sum(total_train_acc) / 5.)
    print("average val accuracy", np.sum(total_val_acc) / 5.)
    
    return 0


In [98]:
def CrossValidation_ConvolutionNetwork(X_train, Y_train, X_test, nFold, verbose=0):
    kf = KFold(n_splits = nFold)
    inds = [ind for ind in kf.split(X_train, Y_train)]
        
    total_train_acc = []
    total_val_acc = []
    
    # Converting Y values to one hot vector
    num_classes = 2
    Y = keras.utils.to_categorical(Y_train, num_classes=num_classes)
    
    
    # For 1D Convolution, change dimension of input 
    X = np.expand_dims(X_train, axis=2)
    
    # perform 5-fold validation
    for i in range(0,nFold):
        traini, vali = inds[i]
        model, prediction, accuracy = ClassifyWithConvolutionNetwork(X_train[traini], Y_train[traini],X_test)
                
        # Compute accuracy
        val_acc = model.evaluate(X[vali], Y[vali], verbose=verbose)

        total_train_acc = np.append(total_train_acc, accuracy)
        total_val_acc = np.append(total_val_acc, val_acc[1])

    print(BORDER)
    print("CROSS VALIDATION: ClassifyWithConvolutionNetwork")
    print(BORDER)
    print("training accuracy", total_train_acc)
    print("val accuracy", total_val_acc)
    print("average training accuracy", np.sum(total_train_acc) / 5.)
    print("average val accuracy", np.sum(total_val_acc) / 5.)
    
    return 0

In [99]:
def CrossValidation_sklearn(modelFunction, X_train, Y_train, X_test, nFold, verbose=0):
    kf = KFold(n_splits = nFold)
    inds = [ind for ind in kf.split(X_train, Y_train)]
        
    total_train_acc = []
    total_val_acc = []
    
    
    # perform 5-fold validation
    for i in range(0,nFold):
        traini, vali = inds[i]
        model, prediction, accuracy = modelFunction(X_train[traini], Y_train[traini], X_test, verbose=verbose)
        
        # Compute accuracy.
        val_acc = model.score(X_train[vali], Y_train[vali])
            
        total_train_acc = np.append(total_train_acc, accuracy)
        total_val_acc = np.append(total_val_acc, val_acc)

    print(BORDER)
    print("CROSS VALIDATION: " + modelFunction.__name__)
    print(BORDER)
    print("training accuracy", total_train_acc)
    print("val accuracy", total_val_acc)
    print("average training accuracy", np.sum(total_train_acc) / float(nFold))
    print("average val accuracy", np.sum(total_val_acc) / float(nFold))
    
    return 0


In [100]:
def CrossValidation_Blending(X_train, Y_train, X_test, nFold, verbose=0):
    nModels = 5
    kf = KFold(n_splits = nFold)
    inds = [ind for ind in kf.split(X_train, Y_train)]
        
    total_train_acc = []
    total_val_acc = []
    
    # perform 5-fold validation
    for i in range(0,nFold):
        traini, vali = inds[i]
        
        model_NB, pred_NB, acc_NB = ClassifyWithBernoulliNB(X_train[traini], Y_train[traini], X_train[vali], verbose=verbose)
        model_LR, pred_LR, acc_LR = ClassifyWithLogisticRegression(X_train[traini], Y_train[traini],X_train[vali], verbose=verbose)
        model_SVM, pred_SVM, acc_SVM = ClassifyWithSVM(X_train[traini], Y_train[traini], X_train[vali], verbose=verbose)
        model_NN, pred_NN, acc_NN = ClassifyWithNeuralNetwork(X_train[traini], Y_train[traini], X_train[vali], verbose=verbose)
        model_CN, pred_CN, acc_CN = ClassifyWithConvolutionNetwork(X_train[traini], Y_train[traini], X_train[vali], verbose=verbose)
        
        
        # Combine prediction into list
        preds = []
        for i in range(len(pred_NN)):
            preds.append((pred_NB[i], pred_NN[i], pred_CN[i], pred_LR[i], pred_SVM[i]))
            
    
        # Final prediction
        Y_pred = np.array([])
        for i in range(len(preds)):
            lst = Counter(preds[i]).most_common()
            Y_pred = np.append(Y_pred, int(lst[0][0]))
            
        total_train_acc = np.append(total_train_acc, acc_NN)
        total_train_acc = np.append(total_train_acc, acc_CN)
        total_train_acc = np.append(total_train_acc, acc_LR)
        total_train_acc = np.append(total_train_acc, acc_SVM)
        total_train_acc = np.append(total_train_acc, acc_NB)
        
        correctlyClassified = (Y_pred == Y_train[vali]).astype(int);
        val_acc = np.sum(correctlyClassified) / correctlyClassified.shape[0]
        total_val_acc = np.append(total_val_acc, val_acc)

    print(BORDER)
    print("CROSS VALIDATION: Blending")
    print(BORDER)
    print("training accuracy", total_train_acc)
    print("val accuracy", total_val_acc)
    print("average training accuracy", np.sum(total_train_acc) / float(nModels * nFold))
    print("average val accuracy", np.sum(total_val_acc) / float(nFold))

In [112]:
def CrossValidation_Stacking(X_train, Y_train, X_test, nFold, verbose=0):
    nModels = 5
    kf = KFold(n_splits = nFold)
    inds = [ind for ind in kf.split(X_train, Y_train)]
        
    total_train_acc = []
    total_val_acc = []
    
    # perform 5-fold validation
    for i in range(0,nFold):
        traini, vali = inds[i]
        model1, pred1, acc1 = ClassifyWithNeuralNetwork(X_train[traini], Y_train[traini], X_train[vali], verbose=verbose)
        model2, pred2, acc2 = ClassifyWithNeuralNetwork(X_train[traini], Y_train[traini], X_train[vali], verbose=verbose)
        model3, pred3, acc3 = ClassifyWithNeuralNetwork(X_train[traini], Y_train[traini], X_train[vali], verbose=verbose)
        model4, pred4, acc4 = ClassifyWithNeuralNetwork(X_train[traini], Y_train[traini], X_train[vali], verbose=verbose)
        model5, pred5, acc5 = ClassifyWithNeuralNetwork(X_train[traini], Y_train[traini], X_train[vali], verbose=verbose)
        
        
        # Combine prediction into list
        preds = []
        for i in range(len(pred1)):
            preds.append((pred1[i], pred2[i], pred3[i], pred4[i], pred5[i]))
            
    
        # Final prediction
        Y_pred = np.array([])
        for i in range(len(preds)):
            lst = Counter(preds[i]).most_common()
            Y_pred = np.append(Y_pred, int(lst[0][0]))
            
        total_train_acc = np.append(total_train_acc, acc1)
        total_train_acc = np.append(total_train_acc, acc2)
        total_train_acc = np.append(total_train_acc, acc3)
        total_train_acc = np.append(total_train_acc, acc4)
        total_train_acc = np.append(total_train_acc, acc5)
        
        correctlyClassified = (Y_pred == Y_train[vali]).astype(int);
        val_acc = np.sum(correctlyClassified) / correctlyClassified.shape[0]
        total_val_acc = np.append(total_val_acc, val_acc)

    print(BORDER)
    print("CROSS VALIDATION: Stacking")
    print(BORDER)
    print("training accuracy", total_train_acc)
    print("val accuracy", total_val_acc)
    print("average training accuracy", np.sum(total_train_acc) / float(nModels * nFold))
    print("average val accuracy", np.sum(total_val_acc) / float(nFold))

In [18]:
# Loads training dataset
training = load_data('data/training_data.txt', 1)
X_train = training[:, 1:]
Y_train = training[:, 0]

In [19]:
# Loads testing dataset
# There is no label for testing set 
X_test = load_data('data/test_data.txt', 1)

In [231]:
# Using Validation   
# CrossValidation_NeuralNetwork(X_train, Y_train, X_test, 5, verbose=1)
# CrossValidation_ConvolutionNetwork(X_train, Y_train, X_test, 5)

# CrossValidation_sklearn(ClassifyWithLinearRegression, X_train, Y_train, X_test, 5)
# CrossValidation_sklearn(ClassifyWithLogisticRegression, X_train, Y_train, X_test, 5)
# CrossValidation_sklearn(ClassifyWithAdaBoost, X_train, Y_train, X_test, 5)
# CrossValidation_sklearn(ClassifyWithDecisionTree, X_train, Y_train, X_test, 5)
# CrossValidation_sklearn(ClassifyWithRandomForest, X_train, Y_train, X_test, 5)

# CrossValidation_sklearn(ClassifyWithSVM, X_train, Y_train, X_test, 5, verbose=1)
# CrossValidation_sklearn(ClassifyWithSVC, X_train, Y_train, X_test, 5, verbose=1)

# CrossValidation_sklearn(ClassifyWithBernoulliNB, X_train, Y_train, X_test, 5)
# CrossValidation_sklearn(ClassifyWithMultinomialNB, X_train, Y_train, X_test, 5)

CrossValidation_Blending(X_train, Y_train, X_test, 5, verbose=1)
# CrossValidation_Stacking(X_train, Y_train, X_test, 5, verbose=1)


Bernoulli Naive Bayes

Training accuracy:  0.825875

LOGISTIC REGRESSION



/Users/Jenny/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Training accuracy:  0.8775

SVM

[LibLinear]Training accuracy:  0.8750625

NEURAL NETWORK

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_577 (Dense)            (None, 128)               128128    
_________________________________________________________________
activation_574 (Activation)  (None, 128)               0         
_________________________________________________________________
dropout_283 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_578 (Dense)            (None, 128)               16512     
_________________________________________________________________
activation_575 (Activation)  (None, 128)               0         
_________________________________________________________________
dropout_284 (Dropout)        (None, 128)               0         
___________________________________________________

Total params: 8,192,706
Trainable params: 8,192,706
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
16000/16000 [==============================] - 114s 7ms/step - loss: 0.4008 - acc: 0.8211
Epoch 2/3
16000/16000 [==============================] - 77s 5ms/step - loss: 0.2939 - acc: 0.8753
Epoch 3/3
16000/16000 [==============================] - 79s 5ms/step - loss: 0.2329 - acc: 0.9049
Training score: 0.14260616794228553
Training accuracy: 0.9515625
4000/4000 [==============================] - 12s 3ms/step

Bernoulli Naive Bayes

Training accuracy:  0.8276875

LOGISTIC REGRESSION

Training accuracy:  0.88

SVM

[LibLinear]Training accuracy:  0.876625

NEURAL NETWORK

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_587 (Dense)            (None, 128)               128128    
_________________________________________________________________
activatio

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_187 (Conv1D)          (None, 1000, 128)         512       
_________________________________________________________________
flatten_68 (Flatten)         (None, 128000)            0         
_________________________________________________________________
dropout_297 (Dropout)        (None, 128000)            0         
_________________________________________________________________
dense_595 (Dense)            (None, 64)                8192064   
_________________________________________________________________
activation_592 (Activation)  (None, 64)                0         
_________________________________________________________________
dropout_298 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_596 (Dense)            (None, 2)                 130       
__________

In [201]:
CrossValidation_Stacking(X_train, Y_train, X_test, 5, verbose=1)


NEURAL NETWORK

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_574 (Dense)            (None, 128)               128128    
_________________________________________________________________
activation_571 (Activation)  (None, 128)               0         
_________________________________________________________________
dropout_281 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_575 (Dense)            (None, 128)               16512     
_________________________________________________________________
activation_572 (Activation)  (None, 128)               0         
_________________________________________________________________
dropout_282 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_576 (Dense)            (None, 2)                 258 

KeyboardInterrupt: 

In [143]:
pred = ClassifyWithBlending(X_train, Y_train, X_test, verbose=1)
WritePredictionOut('Blending', pred)


Blending


NEURAL NETWORK

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_360 (Dense)            (None, 1024)              1025024   
_________________________________________________________________
activation_357 (Activation)  (None, 1024)              0         
_________________________________________________________________
dropout_131 (Dropout)        (None, 1024)              0         
_________________________________________________________________
dense_361 (Dense)            (None, 1024)              1049600   
_________________________________________________________________
activation_358 (Activation)  (None, 1024)              0         
_________________________________________________________________
dropout_132 (Dropout)        (None, 1024)              0         
_________________________________________________________________
dense_362 (Dense)            (None, 2)          

In [144]:
pred = ClassifyWithStacking(X_train, Y_train, X_test, verbose=1)
WritePredictionOut('Stacking', pred)


Blending


NEURAL NETWORK

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_365 (Dense)            (None, 1024)              1025024   
_________________________________________________________________
activation_362 (Activation)  (None, 1024)              0         
_________________________________________________________________
dropout_135 (Dropout)        (None, 1024)              0         
_________________________________________________________________
dense_366 (Dense)            (None, 1024)              1049600   
_________________________________________________________________
activation_363 (Activation)  (None, 1024)              0         
_________________________________________________________________
dropout_136 (Dropout)        (None, 1024)              0         
_________________________________________________________________
dense_367 (Dense)            (None, 2)          

Epoch 1/2
20000/20000 [==============================] - 19s 957us/step - loss: 0.4542 - acc: 0.7851
Epoch 2/2
20000/20000 [==============================] - 10s 495us/step - loss: 0.3074 - acc: 0.8713
Training score: 0.22472439955472945
Training accuracy: 0.9158
10000/10000 [==============================] - 7s 737us/step
[1. 1. 0. ... 0. 1. 0.]
prediction shape is: (10000,)


In [142]:
_, pred, _ = ClassifyWithNeuralNetwork(X_train, Y_train, X_test, verbose=1)
WritePredictionOut('NeuralNet', pred)


NEURAL NETWORK

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_357 (Dense)            (None, 1024)              1025024   
_________________________________________________________________
activation_354 (Activation)  (None, 1024)              0         
_________________________________________________________________
dropout_129 (Dropout)        (None, 1024)              0         
_________________________________________________________________
dense_358 (Dense)            (None, 1024)              1049600   
_________________________________________________________________
activation_355 (Activation)  (None, 1024)              0         
_________________________________________________________________
dropout_130 (Dropout)        (None, 1024)              0         
_________________________________________________________________
dense_359 (Dense)            (None, 2)                 2050

In [192]:
ClassifyWithConvolutionNetwork(X_train, Y_train, X_test, verbose=1)


NEURAL NETWORK

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_178 (Conv1D)          (None, 1000, 128)         512       
_________________________________________________________________
flatten_59 (Flatten)         (None, 128000)            0         
_________________________________________________________________
dropout_269 (Dropout)        (None, 128000)            0         
_________________________________________________________________
dense_562 (Dense)            (None, 64)                8192064   
_________________________________________________________________
activation_559 (Activation)  (None, 64)                0         
_________________________________________________________________
dropout_270 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_563 (Dense)            (None, 2)                 130 

(<keras.models.Sequential at 0x1beb1f07b8>,
 array([1, 1, 0, ..., 0, 1, 0]),
 0.93765)